# Blagodatskaya et al. (2014)

## DEB model calibration

### Importing functions

In [1]:
import numpy as np
import pandas as pd
from scipy.integrate import odeint
from scipy.optimize import dual_annealing
from scipy.optimize import differential_evolution

### This is the DEB model

In [2]:
def DEBmodel (y, t, pars):
    #define initial pools (eu is assumed to be zero)
    S=y[0];    e=y[1];    X1=y[2];     CO2=y[3];
    
    #define parameters
    yA=pars[0]; 
    Km=pars[1];     
    v=pars[2];
    m=pars[3]; 
    g=pars[4]; 
    #k=pars[4];
    ce=pars[5];
    MX1=ce/4;
    
    #Scaling function for substrate uptake
    f=S/(Km+S)
    #Fluxes
    uptake=(v*ce/yA)*X1*f
    growth = (v*e-m*g)/(e + g)
    
    #Define derivatives
    dSdt = -uptake
    dedt = v*(f - e)
    dX1dt = growth*X1 #- k*X1
    dCO2dt = uptake*(1 - yA) + ce*(X1*e*(v-growth)) - growth*X1*MX1 
           
    return dSdt, dedt, dX1dt, dCO2dt;

### The function below uses the output from DEBmodel to convert biomass pools $e$ and $X_{1}$ to microbial biomass ($B$) and measured biomass proxy-parameter DNA 

In [3]:
def calcDEB (model, pars, t, y0):
    #model parameters
    ##yA, Km, v, m, g, ce
    pars_model=pars[0:6]
    #conversion factors
    ##ce, iX1
    conversions=pars[5:7]

    #solve the model
    y=odeint(model,y0,t, args=(pars_model,))

    #calculate biomass (B) and total DNA
    B=(conversions[0]/4 + conversions[0]*y[:, 1])*y[:, 2]
    DNA = 0.25*conversions[1]/(0.25 + y[:, 1])*B
    
    #Create data with predictions
    yhat = np.concatenate((#y[:, 0].reshape(len(d.Time),1),#glucose
                           y[:, 3].reshape(len(d.Time),1),#CO2
                           #Flush.reshape(len(d.Time),1),
                           DNA.reshape(len(d.Time),1)), axis=1)

    return yhat

### Objective function is defined

In [5]:
def obj_funDEB (x):
    #define parameters
    ##yA, Km, v, m, g, ce, iX1
    pars = x

    #initial conditions
    S_i = d.Sinit[0]
    #e_i = d.Cmicinit[0]/d.DNAinit[0]*(pars[7]/4) - (pars[6]/4) is 0
    X1_i = d.DNAinit[0]/(0.25*pars[6]*pars[5])
    
    y0 = np.array([S_i, 0, X1_i, 0])

    #times
    t = d.Time

    #model simulations
    yhat_full = calcDEB(DEBmodel, pars, t, y0)

    #observations
    obs=np.concatenate((#np.array([d.S]).reshape(len(d.Time),1),
                        np.array([d.CO2]).reshape(len(d.Time),1),
                        #np.array([d.Cmic12 + d.Cmic14]).reshape(len(d.Time),1),
                        #np.array([d.Cmic14]).reshape(len(d.Time),1),
                        np.array([d.DNA]).reshape(len(d.Time),1)),
                     axis=1)

    #weights
    weights=np.concatenate((#np.nanstd(d.S).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd(d.CO2).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanstd((d.Cmic12 + d.Cmic14)).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean(d.Cmic14).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd(d.DNA).repeat(len(d.Time)).reshape(len(d.Time),1)),
                       axis=1)


    out=np.nansum(((yhat_full-obs)/weights)**2)

    return out

### Goodness of fit is calculated

In [6]:
def goodnessDEB (x):
    #define parameters
    ##yA, Km, v, m, g, ce, iX1
    pars = x

    #initial conditions
    S_i = d.Sinit[0]
    #e_i = d.Cmicinit[0]/d.DNAinit[0]*(pars[7]/4) - (pars[6]/4) is 0
    X1_i = d.DNAinit[0]/(0.25*pars[6]*pars[5])
    
    y0 = np.array([S_i, 0, X1_i, 0])

    #times
    t = d.Time

    #model simulations
    yhat_full = calcDEB(DEBmodel, pars, t, y0)
    
    #Standardize the simulations
    ##means
    Smeans=np.concatenate((np.nanmean(yhat_full[:, 0]).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanmean(yhat_full[:, 1]).repeat(len(d.Time)).reshape(len(d.Time),1)),
                            #np.nanmean((yhat_full[:, 2])).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean(d.Cmic14).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean(yhat_full[:, 3]).repeat(len(d.Time)).reshape(len(d.Time),1)),
                       axis=1)
    ##std
    Sstd=np.concatenate((np.nanstd(yhat_full[:, 0]).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd(yhat_full[:, 1]).repeat(len(d.Time)).reshape(len(d.Time),1)),
                            #np.nanstd((yhat_full[:, 2])).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean(d.Cmic14).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanstd(yhat_full[:, 3]).repeat(len(d.Time)).reshape(len(d.Time),1)),
                       axis=1)
    

    #observations
    obs=np.concatenate((#np.array([d.S]).reshape(len(d.Time),1),
                        np.array([d.CO2]).reshape(len(d.Time),1),
                        #np.array([d.Cmic12 + d.Cmic14]).reshape(len(d.Time),1),
                        #np.array([d.Cmic14]).reshape(len(d.Time),1),
                        np.array([d.DNA]).reshape(len(d.Time),1)),
                     axis=1)
    #Standardize the observations
    ##means
    Omeans=np.concatenate((#np.nanmean(d.S).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanmean(d.CO2).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean((d.Cmic12 + d.Cmic14)).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean(d.Cmic14).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanmean(d.DNA).repeat(len(d.Time)).reshape(len(d.Time),1)),
                       axis=1)
    ##std
    Ostd=np.concatenate((#np.nanstd(d.S).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd(d.CO2).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanstd((d.Cmic12 + d.Cmic14)).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean(d.Cmic14).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd(d.DNA).repeat(len(d.Time)).reshape(len(d.Time),1)),
                       axis=1)
    
    R2=1-np.nansum((obs-yhat_full)**2)/np.nansum((obs-np.nanmean(obs))**2)
    ll=-np.nansum((obs-yhat_full)**2)/2/np.nanstd(obs)**2
    AIC = len(pars)*2 - 2*ll
    
    #Normalized residual sum of squares 
    Fnorm = np.nansum((((obs-Omeans)/Ostd)-((yhat_full-Smeans)/Sstd))**2)
    
    out = np.array([R2, ll, AIC, Fnorm])

    return out

In [7]:
def predDEB (model, pars, t, y0):
    #model parameters
    ##yA, Km, v, m, g, ce
    pars_model=pars[0:6]
    #conversion factors
    ##ce, iX1
    conversions=pars[5:7]

    #solve the model
    y=odeint(model,y0,t, args=(pars_model,))

    #calculate biomass (B) and total DNA
    B=(conversions[0]/4 + conversions[0]*y[:, 1])*y[:, 2]
    DNA = 0.25*conversions[1]/(0.25 + y[:, 1])*B
    
    #Create data with predictions
    yhat = np.concatenate((#y[:, 0].reshape(len(np.arange(0, 9.05, 0.05)),1),#glucose
                           y[:, 3].reshape(len(np.arange(0, 2.01, 0.01)),1),#CO2
                           #Flush.reshape(len(np.arange(0, 9.05, 0.05)),1),
                           DNA.reshape(len(np.arange(0, 2.01, 0.01)),1)), axis=1)

    return yhat

## Rhizosphere soil

In [27]:
#reading the data
dall = pd.read_csv('/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/SoilMBVariabilityData/Blagodatskaya2014.csv', sep=',')
d = dall[dall.Treatment == "Rhizosphere"]
print(d)

                          Study        Soil Substrate  Clay   pH  Ctot   Ntot  \
0   Blagodatskaya et al. (2014)  Loamy sand   Glucose   NaN  6.7   1.1  0.087   
1   Blagodatskaya et al. (2014)  Loamy sand   Glucose   NaN  6.7   1.1  0.087   
2   Blagodatskaya et al. (2014)  Loamy sand   Glucose   NaN  6.7   1.1  0.087   
3   Blagodatskaya et al. (2014)  Loamy sand   Glucose   NaN  6.7   1.1  0.087   
4   Blagodatskaya et al. (2014)  Loamy sand   Glucose   NaN  6.7   1.1  0.087   
5   Blagodatskaya et al. (2014)  Loamy sand   Glucose   NaN  6.7   1.1  0.087   
6   Blagodatskaya et al. (2014)  Loamy sand   Glucose   NaN  6.7   1.1  0.087   
7   Blagodatskaya et al. (2014)  Loamy sand   Glucose   NaN  6.7   1.1  0.087   
8   Blagodatskaya et al. (2014)  Loamy sand   Glucose   NaN  6.7   1.1  0.087   
9   Blagodatskaya et al. (2014)  Loamy sand   Glucose   NaN  6.7   1.1  0.087   
10  Blagodatskaya et al. (2014)  Loamy sand   Glucose   NaN  6.7   1.1  0.087   
11  Blagodatskaya et al. (20

In [28]:
bla_debparsR=dual_annealing(obj_funDEB, [(0.05, 1), #yA
                                             (10, 10000), #Km
                                             (0.01, 20), #v
                                             (1e-12, 0.1), #m
                                             (0.1, 3), #g
                                             #(1e-12, 0.1), #k
                                             (0.1, 10), #ce
                                             (0, 1)]) #iX1

<ipython-input-5-8302b4cc0558>:9: RuntimeWarning: divide by zero encountered in double_scalars
  X1_i = d.DNAinit[0]/(0.25*pars[6]*pars[5])
<ipython-input-2-c52ab644af63>:25: RuntimeWarning: invalid value encountered in double_scalars
  dCO2dt = uptake*(1 - yA) + ce*(X1*e*(v-growth)) - growth*X1*MX1
/usr/local/lib/python3.8/dist-packages/scipy/integrate/odepack.py:247: ODEintWarning: Illegal input detected (internal error). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
<ipython-input-3-723701b40176>:14: RuntimeWarning: invalid value encountered in multiply
  DNA = 0.25*conversions[1]/(0.25 + y[:, 1])*B


In [29]:
print(bla_debparsR)
print(goodnessDEB(bla_debparsR.x))

     fun: 3.4336435868814874
 message: ['Maximum number of iteration reached']
    nfev: 53649
    nhev: 0
     nit: 1000
    njev: 4956
  status: 0
 success: True
       x: array([1.00000000e+00, 1.83750422e+03, 2.32169356e+00, 1.00000000e-12,
       5.18474772e-01, 1.16188153e+00, 3.60800914e-02])
[ 0.93038251 -1.5663936  17.13278719  2.87709487]


In [30]:
bla_debparsRDE=differential_evolution(obj_funDEB, [(0.05, 1), #yA
                                             (10, 10000), #Km
                                             (0.01, 20), #v
                                             (1e-12, 0.1), #m
                                             (0.1, 3), #g
                                             (0.1, 10), #ce
                                             (0, 1)]) #iX1

<ipython-input-5-8302b4cc0558>:9: RuntimeWarning: divide by zero encountered in double_scalars
  X1_i = d.DNAinit[0]/(0.25*pars[6]*pars[5])
<ipython-input-2-c52ab644af63>:25: RuntimeWarning: invalid value encountered in double_scalars
  dCO2dt = uptake*(1 - yA) + ce*(X1*e*(v-growth)) - growth*X1*MX1
<ipython-input-3-723701b40176>:14: RuntimeWarning: invalid value encountered in multiply
  DNA = 0.25*conversions[1]/(0.25 + y[:, 1])*B


In [31]:
print(bla_debparsRDE)
print(goodnessDEB(bla_debparsRDE.x))

     fun: 3.3502319666487295
     jac: array([-3.77508507e+00, -9.32586601e-06,  4.44089213e-07,  7.09924608e-01,
       -1.33226762e-07,  4.44089213e-08, -8.12683254e-06])
 message: 'Optimization terminated successfully.'
    nfev: 15659
     nit: 146
 success: True
       x: array([1.00000000e+00, 2.08079312e+02, 1.88170413e+00, 1.00000000e-12,
       9.76744681e-01, 5.28801717e+00, 9.62269468e-02])
[ 0.93130046 -1.54573968 17.09147937  3.04504084]


In [32]:
np.savetxt('/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/PythonScripts/bla_debparsR.csv', bla_debparsRDE.x.reshape(1,7), delimiter=",")

## Solution over time for visualization in R

In [33]:
#initial conditions
S_i = d.Sinit[0]
X1_i = d.DNAinit[0]/(0.25*bla_debparsRDE.x[6]*bla_debparsRDE.x[5])
    
y0 = np.array([S_i, 0, X1_i, 0])

#times
t = np.arange(0, 2.01, 0.01)

#model simulations
BlaPredR = predDEB(DEBmodel, bla_debparsRDE.x, t, y0)

## Root free soil

In [34]:
d = dall[dall.Treatment != "Rhizosphere"]
d = d.reset_index()
print(d)

    index                        Study        Soil Substrate  Clay   pH  Ctot  \
0      39  Blagodatskaya et al. (2014)  Loamy sand   Glucose   NaN  6.7   1.1   
1      40  Blagodatskaya et al. (2014)  Loamy sand   Glucose   NaN  6.7   1.1   
2      41  Blagodatskaya et al. (2014)  Loamy sand   Glucose   NaN  6.7   1.1   
3      42  Blagodatskaya et al. (2014)  Loamy sand   Glucose   NaN  6.7   1.1   
4      43  Blagodatskaya et al. (2014)  Loamy sand   Glucose   NaN  6.7   1.1   
5      44  Blagodatskaya et al. (2014)  Loamy sand   Glucose   NaN  6.7   1.1   
6      45  Blagodatskaya et al. (2014)  Loamy sand   Glucose   NaN  6.7   1.1   
7      46  Blagodatskaya et al. (2014)  Loamy sand   Glucose   NaN  6.7   1.1   
8      47  Blagodatskaya et al. (2014)  Loamy sand   Glucose   NaN  6.7   1.1   
9      48  Blagodatskaya et al. (2014)  Loamy sand   Glucose   NaN  6.7   1.1   
10     49  Blagodatskaya et al. (2014)  Loamy sand   Glucose   NaN  6.7   1.1   
11     50  Blagodatskaya et 

In [35]:
bla_debparsS=dual_annealing(obj_funDEB, [(0.05, 1), #yA
                                             (10, 10000), #Km
                                             (0.01, 20), #v
                                             (1e-12, 0.1), #m
                                             (0.1, 3), #g
                                             #(1e-12, 0.1), #k
                                             (0.1, 10), #ce
                                             (0, 1)]) #iX1

<ipython-input-5-8302b4cc0558>:9: RuntimeWarning: divide by zero encountered in double_scalars
  X1_i = d.DNAinit[0]/(0.25*pars[6]*pars[5])
<ipython-input-2-c52ab644af63>:25: RuntimeWarning: invalid value encountered in double_scalars
  dCO2dt = uptake*(1 - yA) + ce*(X1*e*(v-growth)) - growth*X1*MX1
/usr/local/lib/python3.8/dist-packages/scipy/integrate/odepack.py:247: ODEintWarning: Illegal input detected (internal error). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
<ipython-input-3-723701b40176>:14: RuntimeWarning: invalid value encountered in multiply
  DNA = 0.25*conversions[1]/(0.25 + y[:, 1])*B


In [36]:
print(bla_debparsS)
print(goodnessDEB(bla_debparsS.x))

     fun: 4.223509708790268
 message: ['Maximum number of iteration reached']
    nfev: 45721
    nhev: 0
     nit: 1000
    njev: 3965
  status: 0
 success: True
       x: array([1.00000000e+00, 2.06356824e+03, 2.48034117e+00, 1.00000000e-12,
       5.54284843e-01, 6.49570936e+00, 3.50982408e-02])
[ 0.92049442 -1.78887552 17.57775105  2.74150477]


In [37]:
bla_debparsSDE=differential_evolution(obj_funDEB, [(0.05, 1), #yA
                                             (10, 10000), #Km
                                             (0.01, 20), #v
                                             (1e-12, 0.1), #m
                                             (0.1, 3), #g
                                             (0.1, 10), #ce
                                             (0, 1)]) #iX1

<ipython-input-5-8302b4cc0558>:9: RuntimeWarning: divide by zero encountered in double_scalars
  X1_i = d.DNAinit[0]/(0.25*pars[6]*pars[5])
<ipython-input-2-c52ab644af63>:25: RuntimeWarning: invalid value encountered in double_scalars
  dCO2dt = uptake*(1 - yA) + ce*(X1*e*(v-growth)) - growth*X1*MX1
<ipython-input-3-723701b40176>:14: RuntimeWarning: invalid value encountered in multiply
  DNA = 0.25*conversions[1]/(0.25 + y[:, 1])*B


In [38]:
print(bla_debparsSDE)
print(goodnessDEB(bla_debparsSDE.x))

     fun: 3.6541537913780058
     jac: array([-6.41376993e+00,  4.48974154e-04,  3.55271370e-07,  8.18997581e-01,
       -3.99680291e-06,  2.66453528e-07,  1.91402450e-05])
 message: 'Optimization terminated successfully.'
    nfev: 15005
     nit: 140
 success: True
       x: array([1.00000000e+00, 1.22121156e+01, 1.73116518e+00, 1.00000000e-12,
       1.05346187e+00, 1.94362813e+00, 9.14372763e-02])
[ 0.93012171 -1.57226154 17.14452308  2.59638842]


In [39]:
np.savetxt('/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/PythonScripts/bla_debparsS.csv', bla_debparsSDE.x.reshape(1,7), delimiter=",")

## Solution over time for visualization in R

In [40]:
#initial conditions
S_i = d.Sinit[0]
X1_i = d.DNAinit[0]/(0.25*bla_debparsSDE.x[6]*bla_debparsSDE.x[5])
    
y0 = np.array([S_i, 0, X1_i, 0])

#times
t = np.arange(0, 2.01, 0.01)

#model simulations
BlaPredS = predDEB(DEBmodel, bla_debparsSDE.x, t, y0)
Blagodatskaya2014Pred = np.concatenate([BlaPredR, BlaPredS])

In [41]:
np.savetxt('/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/PythonScripts/Blagodatskaya2014Pred.csv', Blagodatskaya2014Pred, delimiter=",")

# Monod and Pirt models are fitted for comparison to DEB

## Monod model
Model as well as supplementary functions are defined below

In [42]:
def Mmodel (y, t, pars):
    #define initial pools
    S=y[0];    B=y[1];    CO2=y[2];
    
    #define parameters
    v=pars[0]; 
    Km=pars[1];     
    CUE=pars[2];
    k=pars[3];
        
    #Fluxes
    uptake = v*S*B/(S + Km)
    growth = uptake*CUE
    respiration = uptake*(1 - CUE)
    death = B*k
    
    #Define derivatives
    dSdt = -uptake
    dBdt = growth - death
    dCO2dt = respiration
           
    return dSdt, dBdt, dCO2dt;

In [43]:
def calcM (model, pars, t, y0):
    #model parameters
    ##v, Km, CUE, k
    pars_model=pars[0:4]
    #conversion factors
    ##kdna
    conversion=pars[4]

    #solve the model
    y=odeint(model,y0,t, args=(pars_model,))

    #calculate total DNA, and flush (Flush)
    #Flush = conversions[0]*y[:, 1]
    DNA = conversion*y[:, 1]
    
    #Create data with predictions
    yhat = np.concatenate((#y[:, 0].reshape(len(d.Time),1),#glucose
                           y[:, 2].reshape(len(d.Time),1),#CO2
                           #Flush.reshape(len(d.Time),1),
                           DNA.reshape(len(d.Time),1)), axis=1)

    return yhat

In [52]:
def obj_funM (x):
    #define parameters
    ##v, Km, CUE, k, kdna
    pars = x

    #initial conditions
    S_i = d.Sinit[0]
    B_i = d.DNAinit[0]/pars[4]
    
    y0 = np.array([S_i, B_i, 0])

    #times
    t = d.Time

    #model simulations
    yhat_full = calcM(Mmodel, pars, t, y0)

    #observations
    obs=np.concatenate((#np.array([d.S]).reshape(len(d.Time),1),
                        np.array([d.CO2]).reshape(len(d.Time),1),
                        #np.array([d.Cmic12 + d.Cmic14]).reshape(len(d.Time),1),
                        #np.array([d.Cmic14]).reshape(len(d.Time),1),
                        np.array([d.DNA]).reshape(len(d.Time),1)),
                     axis=1)

    #weights
    weights=np.concatenate((#np.nanstd(d.S).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd(d.CO2).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanstd((d.Cmic12 + d.Cmic14)).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean(d.Cmic14).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd(d.DNA).repeat(len(d.Time)).reshape(len(d.Time),1)),
                       axis=1)


    out=np.nansum(((yhat_full-obs)/weights)**2)

    return out

In [50]:
def goodnessM (x):
    #define parameters
    ##v, Km, CUE, k, kdna
    pars = x

    #initial conditions
    S_i = d.Sinit[0]
    B_i = d.DNAinit[0]/pars[4]
    
    y0 = np.array([S_i, B_i, 0])

    #times
    t = d.Time

    #model simulations
    yhat_full = calcM(Mmodel, pars, t, y0)

    #Standardize the simulations
    ##means
    Smeans=np.concatenate((np.nanmean(yhat_full[:, 0]).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanmean(yhat_full[:, 1]).repeat(len(d.Time)).reshape(len(d.Time),1)),
                            #np.nanmean((yhat_full[:, 2])).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean(d.Cmic14).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean(yhat_full[:, 3]).repeat(len(d.Time)).reshape(len(d.Time),1)),
                       axis=1)
    ##std
    Sstd=np.concatenate((np.nanstd(yhat_full[:, 0]).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd(yhat_full[:, 1]).repeat(len(d.Time)).reshape(len(d.Time),1)),
                            #np.nanstd((yhat_full[:, 2])).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean(d.Cmic14).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanstd(yhat_full[:, 3]).repeat(len(d.Time)).reshape(len(d.Time),1)),
                       axis=1)
    

    #observations
    obs=np.concatenate((#np.array([d.S]).reshape(len(d.Time),1),
                        np.array([d.CO2]).reshape(len(d.Time),1),
                        #np.array([d.Cmic12 + d.Cmic14]).reshape(len(d.Time),1),
                        #np.array([d.Cmic14]).reshape(len(d.Time),1),
                        np.array([d.DNA]).reshape(len(d.Time),1)),
                     axis=1)
    #Standardize the observations
    ##means
    Omeans=np.concatenate((#np.nanmean(d.S).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanmean(d.CO2).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean((d.Cmic12 + d.Cmic14)).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean(d.Cmic14).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanmean(d.DNA).repeat(len(d.Time)).reshape(len(d.Time),1)),
                       axis=1)
    ##std
    Ostd=np.concatenate((#np.nanstd(d.S).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd(d.CO2).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanstd((d.Cmic12 + d.Cmic14)).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean(d.Cmic14).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd(d.DNA).repeat(len(d.Time)).reshape(len(d.Time),1)),
                       axis=1)
    
    R2=1-np.nansum((obs-yhat_full)**2)/np.nansum((obs-np.nanmean(obs))**2)
    ll=-np.nansum((obs-yhat_full)**2)/2/np.nanstd(obs)**2
    AIC = len(pars)*2 - 2*ll
    
    #Normalized residual sum of squares 
    Fnorm = np.nansum((((obs-Omeans)/Ostd)-((yhat_full-Smeans)/Sstd))**2)
    
    out = np.array([R2, ll, AIC, Fnorm])

    return out

In [46]:
def predM (model, pars, t, y0):
    #model parameters
    ##v, Km, CUE, k
    pars_model=pars[0:4]
    #conversion factors
    ##kdna
    conversion=pars[4]

    #solve the model
    y=odeint(model,y0,t, args=(pars_model,))

    #calculate total DNA, and flush (Flush)
    #Flush = conversions[0]*y[:, 1]
    DNA = conversion*y[:, 1]
    
    #Create data with predictions
    yhat = np.concatenate((#y[:, 0].reshape(len(np.arange(0, 9.05, 0.05)),1),#glucose
                           y[:, 2].reshape(len(np.arange(0, 2.01, 0.01)),1),#CO2
                           #Flush.reshape(len(np.arange(0, 9.05, 0.05)),1),
                           DNA.reshape(len(np.arange(0, 2.01, 0.01)),1)), axis=1)

    return yhat

## Rhizosphere soil

In [47]:
d = dall[dall.Treatment == "Rhizosphere"]

In [55]:
bla_mparsR=dual_annealing(obj_funM, [(0.001, 100), #v
                                             (0.1, 10000), #Km                                             
                                             (0, 1), #CUE
                                             (1e-15, 0.1), #k
                                             #(0, 1), #kec
                                             (0, 1)]) #kDNA
                                             #(0, 1)]) #iX1

<ipython-input-52-aac74c37740c>:8: RuntimeWarning: divide by zero encountered in double_scalars
  B_i = d.DNAinit[0]/pars[4]
<ipython-input-42-15e34bf3de91>:13: RuntimeWarning: invalid value encountered in double_scalars
  growth = uptake*CUE
<ipython-input-43-c272d95c6c09>:14: RuntimeWarning: invalid value encountered in multiply
  DNA = conversion*y[:, 1]
<ipython-input-42-15e34bf3de91>:19: RuntimeWarning: invalid value encountered in double_scalars
  dBdt = growth - death


In [56]:
print(bla_mparsR)
print(goodnessM(bla_mparsR.x))

     fun: 6.332862994272612
 message: ['Maximum number of iteration reached']
    nfev: 22157
    nhev: 0
     nit: 1000
    njev: 2026
  status: 0
 success: True
       x: array([9.17374255e+01, 7.85410590e+03, 7.49905201e-02, 1.00000000e-15,
       4.45873187e-01])
[ 0.87720004 -2.76299906 15.52599812  3.75097875]


In [58]:
#initial conditions
S_i = d.Sinit[0]
B_i = d.DNAinit[0]/bla_mparsR.x[4]
    
y0 = np.array([S_i, B_i, 0])

#times
t = np.arange(0, 2.01, 0.01)

#model simulations
blaRM = predM(Mmodel, bla_mparsR.x, t, y0)

## Root free soil

In [59]:
d = dall[dall.Treatment != "Rhizosphere"]
d = d.reset_index()

In [60]:
bla_mparsS=dual_annealing(obj_funM, [(0.001, 100), #v
                                             (0.1, 10000), #Km                                             
                                             (0, 1), #CUE
                                             (1e-15, 0.1), #k
                                             #(0, 1), #kec
                                             (0, 1)]) #kDNA
                                             #(0, 1)]) #iX1

<ipython-input-52-aac74c37740c>:8: RuntimeWarning: divide by zero encountered in double_scalars
  B_i = d.DNAinit[0]/pars[4]
<ipython-input-42-15e34bf3de91>:13: RuntimeWarning: invalid value encountered in double_scalars
  growth = uptake*CUE
<ipython-input-43-c272d95c6c09>:14: RuntimeWarning: invalid value encountered in multiply
  DNA = conversion*y[:, 1]
<ipython-input-42-15e34bf3de91>:19: RuntimeWarning: invalid value encountered in double_scalars
  dBdt = growth - death


In [61]:
print(bla_mparsS)
print(goodnessM(bla_mparsS.x))

     fun: 7.893155841114035
 message: ['Maximum number of iteration reached']
    nfev: 22163
    nhev: 0
     nit: 1000
    njev: 2027
  status: 0
 success: True
       x: array([6.85761122e+01, 2.24277673e+03, 3.46580796e-02, 1.00000000e-15,
       8.68986782e-01])
[ 0.85406413 -3.28355719 16.56711437  4.18352029]


In [62]:
#initial conditions
S_i = d.Sinit[0]
B_i = d.DNAinit[0]/bla_mparsS.x[4]
    
y0 = np.array([S_i, B_i, 0])

#times
t = np.arange(0, 2.01, 0.01)

#model simulations
blaSM = predM(Mmodel, bla_mparsS.x, t, y0)
Blagodatskaya2014PredM = np.concatenate([blaRM, blaSM])
np.savetxt('/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/PythonScripts/Blagodatskaya2014PredM.csv', Blagodatskaya2014PredM, delimiter=",")

## Pirt model
Model as well as supplementary functions are defined below

In [63]:
def Pmodel (y, t, pars):
    #define initial pools
    S=y[0];    B=y[1];    CO2=y[2];
    
    #define parameters
    v=pars[0]; 
    Km=pars[1];     
    CUE=pars[2];
    m = pars[3];
    k = pars[4];
        
    #Fluxes
    uptake = v*S*B/(S + Km)
    growth = uptake*CUE
    respiration = uptake*(1 - CUE) + B*m
    death = B*k
    
    #Define derivatives
    dSdt = -uptake
    dBdt = growth - death
    dCO2dt = respiration
           
    return dSdt, dBdt, dCO2dt;

In [64]:
def calcP (model, pars, t, y0):
    #model parameters
    ##v, Km, CUE, m, k
    pars_model=pars[0:5]
    #conversion factors
    ##kdna
    conversion=pars[5]

    #solve the model
    y=odeint(model,y0,t, args=(pars_model,))

    #calculate total DNA, and flush (Flush)
    #Flush = conversions[0]*y[:, 1]
    DNA = conversion*y[:, 1]
    
    #Create data with predictions
    yhat = np.concatenate((#y[:, 0].reshape(len(d.Time),1),#glucose
                           y[:, 2].reshape(len(d.Time),1),#CO2
                           #Flush.reshape(len(d.Time),1),
                           DNA.reshape(len(d.Time),1)), axis=1)

    return yhat

In [65]:
def obj_funP (x):
    #define parameters
    ##v, Km, CUE, m, k, kdna
    pars = x

    #initial conditions
    S_i = d.Sinit[0]
    B_i = d.DNAinit[0]/pars[5]
    
    y0 = np.array([S_i, B_i, 0])

    #times
    t = d.Time

    #model simulations
    yhat_full = calcP(Pmodel, pars, t, y0)

    #observations
    obs=np.concatenate((#np.array([d.S]).reshape(len(d.Time),1),
                        np.array([d.CO2]).reshape(len(d.Time),1),
                        #np.array([d.Cmic12 + d.Cmic14]).reshape(len(d.Time),1),
                        #np.array([d.Cmic14]).reshape(len(d.Time),1),
                        np.array([d.DNA]).reshape(len(d.Time),1)),
                     axis=1)

    #weights
    weights=np.concatenate((#np.nanstd(d.S).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd(d.CO2).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanstd((d.Cmic12 + d.Cmic14)).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean(d.Cmic14).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd(d.DNA).repeat(len(d.Time)).reshape(len(d.Time),1)),
                       axis=1)


    out=np.nansum(((yhat_full-obs)/weights)**2)

    return out

In [66]:
def goodnessP (x):
    #define parameters
    ##v, Km, CUE, k, kdna
    pars = x

    #initial conditions
    S_i = d.Sinit[0]
    B_i = d.DNAinit[0]/pars[5]
    
    y0 = np.array([S_i, B_i, 0])

    #times
    t = d.Time

    #model simulations
    yhat_full = calcP(Pmodel, pars, t, y0)

    #Standardize the simulations
    ##means
    Smeans=np.concatenate((np.nanmean(yhat_full[:, 0]).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanmean(yhat_full[:, 1]).repeat(len(d.Time)).reshape(len(d.Time),1)),
                            #np.nanmean((yhat_full[:, 2])).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean(d.Cmic14).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean(yhat_full[:, 3]).repeat(len(d.Time)).reshape(len(d.Time),1)),
                       axis=1)
    ##std
    Sstd=np.concatenate((np.nanstd(yhat_full[:, 0]).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd(yhat_full[:, 1]).repeat(len(d.Time)).reshape(len(d.Time),1)),
                            #np.nanstd((yhat_full[:, 2])).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean(d.Cmic14).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanstd(yhat_full[:, 3]).repeat(len(d.Time)).reshape(len(d.Time),1)),
                       axis=1)
    

    #observations
    obs=np.concatenate((#np.array([d.S]).reshape(len(d.Time),1),
                        np.array([d.CO2]).reshape(len(d.Time),1),
                        #np.array([d.Cmic12 + d.Cmic14]).reshape(len(d.Time),1),
                        #np.array([d.Cmic14]).reshape(len(d.Time),1),
                        np.array([d.DNA]).reshape(len(d.Time),1)),
                     axis=1)
    #Standardize the observations
    ##means
    Omeans=np.concatenate((#np.nanmean(d.S).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanmean(d.CO2).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean((d.Cmic12 + d.Cmic14)).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean(d.Cmic14).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanmean(d.DNA).repeat(len(d.Time)).reshape(len(d.Time),1)),
                       axis=1)
    ##std
    Ostd=np.concatenate((#np.nanstd(d.S).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd(d.CO2).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanstd((d.Cmic12 + d.Cmic14)).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean(d.Cmic14).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd(d.DNA).repeat(len(d.Time)).reshape(len(d.Time),1)),
                       axis=1)
    
    R2=1-np.nansum((obs-yhat_full)**2)/np.nansum((obs-np.nanmean(obs))**2)
    ll=-np.nansum((obs-yhat_full)**2)/2/np.nanstd(obs)**2
    AIC = len(pars)*2 - 2*ll
    
    #Normalized residual sum of squares 
    Fnorm = np.nansum((((obs-Omeans)/Ostd)-((yhat_full-Smeans)/Sstd))**2)
    
    out = np.array([R2, ll, AIC, Fnorm])

    return out

In [67]:
def predP (model, pars, t, y0):
    #model parameters
    ##v, Km, CUE, m, k
    pars_model=pars[0:5]
    #conversion factors
    ##kdna
    conversion=pars[5]

    #solve the model
    y=odeint(model,y0,t, args=(pars_model,))

    #calculate total DNA, and flush (Flush)
    #Flush = conversions[0]*y[:, 1]
    DNA = conversion*y[:, 1]
    
    #Create data with predictions
    yhat = np.concatenate((#y[:, 0].reshape(len(np.arange(0, 9.05, 0.05)),1),#glucose
                           y[:, 2].reshape(len(np.arange(0, 2.01, 0.01)),1),#CO2
                           #Flush.reshape(len(np.arange(0, 9.05, 0.05)),1),
                           DNA.reshape(len(np.arange(0, 2.01, 0.01)),1)), axis=1)

    return yhat

## Rhizosphere soil

In [68]:
d = dall[dall.Treatment == "Rhizosphere"]

In [71]:
bla_pparsR=dual_annealing(obj_funP, [(0.001, 100), #v
                                             (0.1, 10000), #Km                                             
                                             (0, 1), #CUE
                                             (1e-15, 0.1), #k
                                             (1e-15, 0.1), #m
                                             #(0, 1), #kec
                                             (0, 1)]) #kDNA
                                             #(0, 1)]) #iX1

<ipython-input-65-47133bf34c73>:8: RuntimeWarning: divide by zero encountered in double_scalars
  B_i = d.DNAinit[0]/pars[5]
<ipython-input-63-c3d2819ce9aa>:14: RuntimeWarning: invalid value encountered in double_scalars
  growth = uptake*CUE
<ipython-input-64-5a4a2a03f6b8>:14: RuntimeWarning: invalid value encountered in multiply
  DNA = conversion*y[:, 1]
<ipython-input-63-c3d2819ce9aa>:20: RuntimeWarning: invalid value encountered in double_scalars
  dBdt = growth - death
<ipython-input-63-c3d2819ce9aa>:15: RuntimeWarning: invalid value encountered in double_scalars
  respiration = uptake*(1 - CUE) + B*m


In [72]:
print(bla_pparsR)
print(goodnessP(bla_pparsR.x))

     fun: 6.329987219711947
 message: ['Maximum number of iteration reached']
    nfev: 26995
    nhev: 0
     nit: 1000
    njev: 2142
  status: 0
 success: True
       x: array([7.59518790e+01, 8.10945204e+03, 9.32491598e-02, 1.00000000e-01,
       1.00000000e-15, 3.57155203e-01])
[ 0.877268   -2.76147002 17.52294004  3.75062408]


In [73]:
#initial conditions
S_i = d.Sinit[0]
B_i = d.DNAinit[0]/bla_pparsR.x[5]
    
y0 = np.array([S_i, B_i, 0])

#times
t = np.arange(0, 2.01, 0.01)

#model simulations
blaPR = predP(Pmodel, bla_pparsR.x, t, y0)

## Root free soil

In [74]:
d = dall[dall.Treatment != "Rhizosphere"]
d = d.reset_index()

In [75]:
bla_pparsS=dual_annealing(obj_funP, [(0.001, 100), #v
                                             (0.1, 10000), #Km                                             
                                             (0, 1), #CUE
                                             (1e-15, 0.1), #k
                                             (1e-15, 0.1), #m
                                             #(0, 1), #kec
                                             (0, 1)]) #kDNA
                                             #(0, 1)]) #iX1

<ipython-input-65-47133bf34c73>:8: RuntimeWarning: divide by zero encountered in double_scalars
  B_i = d.DNAinit[0]/pars[5]
<ipython-input-63-c3d2819ce9aa>:14: RuntimeWarning: invalid value encountered in double_scalars
  growth = uptake*CUE
<ipython-input-64-5a4a2a03f6b8>:14: RuntimeWarning: invalid value encountered in multiply
  DNA = conversion*y[:, 1]
<ipython-input-63-c3d2819ce9aa>:20: RuntimeWarning: invalid value encountered in double_scalars
  dBdt = growth - death


In [76]:
print(bla_pparsS)
print(goodnessP(bla_pparsS.x))

     fun: 7.9505052386438075
 message: ['Maximum number of iteration reached']
    nfev: 24188
    nhev: 0
     nit: 1000
    njev: 1741
  status: 0
 success: True
       x: array([9.17261283e+01, 3.83210356e+03, 3.94698748e-02, 1.00000000e-01,
       1.00000000e-15, 7.64725318e-01])
[ 0.85301171 -3.30723653 18.61447307  4.21367685]


In [77]:
#initial conditions
S_i = d.Sinit[0]
B_i = d.DNAinit[0]/bla_pparsS.x[5]
    
y0 = np.array([S_i, B_i, 0])

#times
t = np.arange(0, 2.01, 0.01)

#model simulations
blaPS = predP(Pmodel, bla_pparsS.x, t, y0)

Blagodatskaya2014PredP = np.concatenate([blaPR, blaPS])
np.savetxt('/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/PythonScripts/Blagodatskaya2014PredP.csv', Blagodatskaya2014PredP, delimiter=",")